# Interactive Visualization

In this lecture we will use [**Bokeh**](http://docs.bokeh.org/en/latest/) and [**HoloViews**](https://holoviews.org/), which are some of the newest libraries so make sure your version and dependencies are up to date. The recommended way to install/update the library (and all libraries that it depends on) is to run the following command in the anaconda command prompt:
> conda install -c pyviz holoviews bokeh

# Visualizing Geo Data

*Wikipedia:* A **choropleth map** (from Greek χῶρος ("area/region") + πλῆθος ("multitude")) is a thematic map in which areas are shaded or patterned in proportion to the measurement of the statistical variable being displayed on the map, such as population density or per-capita income.

Choropleth maps provide an easy way to visualize how a measurement varies across a geographic area or show the level of variability within a region. 

We will use [HoloViews](http://holoviews.org/gallery/demos/bokeh/texas_choropleth_example.html) to create choropleth in Jupyter Notebooks.

## Getting Started

### Load Libraries

In [ ]:
import pandas as pd
import numpy as np
import holoviews as hv
hv.notebook_extension('bokeh')

In [ ]:
import bokeh.sampledata

### Get Data

In [ ]:
from bokeh.sampledata.us_states import data as states

state_xs = [states[code]["lons"] for code in states if code not in ['HI','AK']]
state_ys = [states[code]["lats"] for code in states if code not in ['HI','AK']]

## Create a basic map

In [ ]:
from bokeh.plotting import figure, output_notebook, show
p = figure(title="US Map", toolbar_location="left",plot_width=800, plot_height=500)
p.patches(state_xs, state_ys, fill_alpha=0.0,line_color="#884444", line_width=2, line_alpha=0.3)
output_notebook()
show(p)

### Customizing Maps

In [ ]:
dat = list(states.values())
out=[] #<------ this is where I will store the data for visualization
for d in dat:
    if d['name'] not in ['Alaska','Hawaii']:
        d['randomNumber']=np.random.random()
        d['someText']="ABC_"+str(d['name'])
        d['averageLon']=np.abs(np.nanmean(d['lons'])/200) 
        out.append(d)

In [ ]:
np.nanmean(d['lons'])

In [ ]:
choropleth = hv.Polygons(out, ['lons', 'lats'], [('name', 'State'),('averageLon','averageLon'),('someText','TEXT')]) #(key in dat, display name)
choropleth.options(logz=True, tools=['hover'], xaxis=None, yaxis=None,
                   show_grid=False, show_frame=False, width=575, height=400,
                   colorbar=False, toolbar='above', line_color='black',
                   color_index='averageLon',cmap="gray") #more cmaps: https://matplotlib.org/examples/color/colormaps_reference.html

In [ ]:
choropleth = hv.Polygons(out, ['lons', 'lats']) #(key in dat, display name)
choropleth.options(logz=True, cmap="gray")

## Another Example

In [ ]:
#Get data from csv files
import pandas as pd
unEmp = pd.read_csv("data_files\\unemployment09.csv",header=None)
counties = pd.read_csv("data_files\\us_counties.csv")
merged_data=counties.merge(unEmp,left_on=["STATE num","COUNTY num"],right_on=[1,2])
indiana = merged_data[merged_data['State Abbr.']=='IN']

In [ ]:
def get_lons(entry):
    step1=entry.split("<coordinates>")[1].split("</coordinates>")[0].split(" ")
    return [float(x.split(",")[0]) for x in step1]

def get_lats(entry):
    step1=entry.split("<coordinates>")[1].split("</coordinates>")[0].split(" ")
    return [float(x.split(",")[1]) for x in step1]

def get_data(row):
    return dict(name=row['County Name'],
            lons=get_lons(row['geometry']),
            lats=get_lats(row['geometry']),
            unemployment=row[8])

In [ ]:
#organize data into a format that holoviews polygos can visualize
data = [get_data(row) for i,row in indiana.iterrows()]

In [ ]:
choropleth = hv.Polygons(data, ['lons', 'lats'], [('name', 'County'), ('unemployment','Unemployment')]) #(name in data, display name)
plot = choropleth.options(logz=True, tools=['hover'], xaxis=None, yaxis=None,
                   show_grid=False, show_frame=False, width=275, height=400,
                   color_index='Unemployment', colorbar=False, toolbar='above', line_color='black')

In [ ]:
hv.save(plot,"output_files\\s75_example_indiana_counties",fmt='html')